<a href="https://colab.research.google.com/github/vmotta/Algebra-Linear-Python/blob/master/geracao_pb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""TensorFlow.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/vmotta/Algebra-Linear-Python/blob/master/TensorFlow.ipynb

Teste do tensorflow
"""

# This file is useful for reading the contents of the ops generated by ruby.
# You can read any graph defination in pb/pbtxt format generated by ruby
# or by python and then convert it back and forth from human readable to binary format.

!pip install tensorflow==1.4.0
import tensorflow as tf
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

from google.protobuf import text_format

def pbtxt_to_graphdef(filename,save_dir,filename2):
    with open(filename, 'r') as f:
        graph_def = tf.GraphDef()
        file_content = f.read()
        text_format.Merge(file_content, graph_def)
        tf.import_graph_def(graph_def, name='')
        tf.train.write_graph(graph_def, save_dir+'pbtxt/', filename2+'.pb', as_text=False)
    
import numpy as np
import os
import time
#criação da base de dados
x=np.array([[0,0],[0,1],[1,0],[1,1]])#atributos previsores
y=np.array([[0],[1],[1],[0]])#atributis da classe

#definição
neuronios_entrada = 2
neuronios_oculta = 3
neuronios_saida = 1

#criação dos pesos
w = {'oculta' : tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta]), name='w_oculta'),
    'saida': tf.Variable(tf.random_normal([neuronios_oculta,neuronios_saida]), name='w_saida')}

#criação do bias
b = {'oculta' : tf.Variable(tf.random_normal([neuronios_oculta]), name='b_oculta'),
    'saida': tf.Variable(tf.random_normal([neuronios_saida]), name='b_saida')}

#criação dos placesholders
xph = tf.placeholder(tf.float32,[4, neuronios_entrada], name='xph')
yph = tf.placeholder(tf.float32,[4, neuronios_saida], name='yph')

#formulas
camada_oculta = tf.add(tf.matmul(xph,w['oculta']),b['oculta'])
camada_oculta_ativacao = tf.sigmoid(camada_oculta)
camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']),b['saida'])
camada_saida_ativacao = tf.sigmoid(camada_saida)
erro = tf.losses.mean_squared_error(yph, camada_saida_ativacao)
otimizador = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(erro)


#save_dir = './checkpoints/'
save_graph = '/content/graphs/'

#if not os.path.exists(save_dir):
#    os.makedirs(save_dir)
    
if not os.path.exists(save_graph):
    os.makedirs(save_graph)
    
init = tf.global_variables_initializer()#cria init para iniciar variáveis do tensorflow

#inicia sessão tensorflow
with tf.Session() as sess:
    sess.run(init)
    epocas=0
    erro_medio = 1
    while erro_medio >= 1e-5:
        erro_medio = 0
        _, custo =sess.run([otimizador, erro], feed_dict = {xph: x, yph: y})
        if (epocas % 200 == 0) or (erro_medio == 0):
            erro_medio += custo/4
            epocas+=1
    writer = tf.summary.FileWriter(save_graph, sess.graph)
    #tf.io.write_graph(sess.graph, 'graphs/', 'my_test_model.pbtxt')
    tf.train.write_graph(sess.graph_def, 'graphs/', 'my_test_model.pbtxt') #output in pbtxt format
    
    w_final, b_final = sess.run([w,b])    
    filename = 'my_test_model'
    saver = tf.train.Saver()
    save_path = saver.save(sess, "graphs/my_test_model.cpkt")
    print('done saving at',save_path)
    print(save_graph + filename + '.pbtxt')
    pbtxt_to_graphdef(save_graph + filename + '.pbtxt', save_graph,filename)
    #print(sess.graph)
    

#formulas teste de rede
camada_oculta_teste = tf.add(tf.matmul(xph,w_final['oculta']),b_final['oculta'])
camada_oculta_ativacao_teste = tf.sigmoid(camada_oculta_teste)
camada_saida_teste = tf.add(tf.matmul(camada_oculta_ativacao_teste, w_final['saida']),b_final['saida'])
camada_saida_ativacao_teste = tf.sigmoid(camada_saida_teste)
#print(camada_saida_ativacao_teste)

feed_dict_test = {xph: x, yph: y}
with tf.Session() as sess:
    sess.run(init)
    y_pred = sess.run(camada_saida_ativacao_teste, feed_dict=feed_dict_test)
    print(y_pred)
    # Export checkpoint to SavedModel
    train_writer = tf.summary.FileWriter('model/', sess.graph)
    tf.train.write_graph(tf.GraphDef(), 'model/','saved_model.pb', as_text=False)
    




In [19]:
import tensorflow as tf
from tensorflow.python.platform import gfile
!pip install tensorboardcolab
#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
#tbc=TensorBoardColab()
with tf.Session() as sess:
    model_filename ='model/saved_model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='model/'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)
#summary_writer = tbc.get_writer()
%load_ext tensorboard
%tensorboard --logdir ./graphs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 6, in <module>
    from tensorboard.main import main
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 31, in <module>
    from tensorboard import program
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 43, in <module>
    from tensorboard.backend import application
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/backend/application.py", line 38, in <module>
    from tensorboard import db
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/db.py", line 42, in <module>
    class TensorBase(object):
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/db.py", lin